In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams 
from sklearn import ensemble
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
import xgboost as xgb

eng_stopwords = set(stopwords.words('english'))

from pandas import DataFrame

E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# *Prepare data*

In [2]:
train1 = pd.read_csv("../train/train1.csv")
test1 = pd.read_csv("../test/test1.csv")
print("train1:",train1.shape)
print("test1:",test1.shape)

train2 = pd.read_csv("../train/train2.csv")
test2 = pd.read_csv("../test/test2.csv")
print("train2:",train2.shape)
print("test2:",test2.shape)

train3 = pd.read_csv("../train/train3.csv")
test3 = pd.read_csv("../test/test3.csv")
print("train3:",train3.shape)
print("test3:",test3.shape)

train4 = pd.read_csv("../train/train4.csv")
test4 = pd.read_csv("../test/test4.csv")
print("train4:",train4.shape)
print("test4:",test4.shape)

train1: (404290, 8)
test1: (2345796, 5)
train2: (404290, 7)
test2: (2345796, 7)
train3: (404290, 6)
test3: (2345796, 5)
train4: (404290, 2)
test4: (2345796, 2)


In [11]:
train1.head()

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,0.772164
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,0.361758
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,0.355191
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,0.000000


In [10]:
test1.head()

,test_id,question1,question2,word_match,tfidf_word_match
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,0.266667,0.274019
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,0.500000,0.480962
2,2,What but is the best way to send money from Ch...,What you send money to China?,0.444444,0.468893
3,3,Which food not emulsifiers?,What foods fibre?,0.000000,0.000000
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,0.800000,1.000000


In [5]:
train1['xxx']=train1.qid1-train1.qid2
train1.head()

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match,xxx
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,0.772164,-1
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,0.361758,-1
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,0.355191,-1
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.000000,-1
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,0.000000,-1


In [6]:
train1[(train1["xxx"]!=-1)]#184930 rows × 9 columns

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match,xxx
264,264,81,529,Why do Slavs squat?,Do squats work for men?,0,0.000000,0.000000,-448
326,326,652,108,What are the most interesting foods you have e...,What is the most delicious dish you've ever ea...,1,0.200000,0.233642,544
353,353,705,134,What are the most important books ever written?,What is the most important book you have ever ...,1,0.500000,0.433054,571
647,647,1292,1127,How do I make money with YouTube?,How do I make money through YouTube?,1,1.000000,1.000000,165
704,704,1189,1405,When will the best time for having sex?,What would be the best time to have sex?,1,0.857143,0.909044,-216
729,729,883,1454,Why is Saltwater taffy candy imported in China?,Why is Saltwater taffy candy imported in Laos?,1,0.800000,0.799245,-571
1027,1027,2049,1919,What food should I eat to gain weight?,How could I gain weight in a healthy way?,0,0.222222,0.236910,130
1081,1081,1863,2156,Did Trump win the election?,What do you think of Trump winning the elections?,0,0.285714,0.219689,-293
1114,1114,2221,1093,How can I hack a WhatsApp account?,How can I hack another WhatsApp account?,1,0.857143,0.855327,1128
1332,1332,2656,1215,Is a third world war coming?,Is World War 3 more imminent than expected?,1,0.444444,0.365184,1441


In [32]:
#train2.drop(['question1','question2'],axis=1, inplace=True)
#test2.drop(['question1','question2'],axis=1, inplace=True)

train_df.drop(['qid1','qid2','question1','question2'],axis=1, inplace=True)
test_df.drop(['question1','question2'],axis=1, inplace=True)

In [6]:
train3=train3[['id','q1_freq','q2_freq']]

In [7]:
train_df=pd.merge(train1,train2,how='inner', on='id') 
train_df=pd.merge(train_df,train3,how='inner', on='id') 
train_df=pd.merge(train_df,train4,how='inner', on='id') 
train_df

,id,qid1,qid2,question1,question2,is_duplicate,word_match,tfidf_word_match,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio,q1_freq,q2_freq,q1_q2_intersect
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,0.772164,6.0,0.857143,5.0,0.625000,4.0,0.571429,1,2,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,0.361758,6.0,0.461538,4.0,0.307692,3.0,0.250000,8,3,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,0.355191,3.0,0.300000,0.0,0.000000,0.0,0.000000,2,1,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0.000000,1.0,0.062500,0.0,0.000000,0.0,0.000000,1,1,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,0.000000,3.0,0.200000,0.0,0.000000,0.0,0.000000,3,1,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,0.470588,0.510771,5.0,0.312500,1.0,0.050000,0.0,0.000000,1,1,0
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,0.000000,0.000000,1.0,0.100000,0.0,0.000000,0.0,0.000000,1,1,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,0.500000,0.645836,2.0,0.500000,1.0,0.333333,0.0,0.000000,1,1,0
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,0.500000,0.396755,3.0,0.375000,0.0,0.000000,0.0,0.000000,2,3,1
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,0.363636,0.503203,4.0,0.333333,0.0,0.000000,0.0,0.000000,1,1,0


In [14]:
#train3=train3[['id','q1_freq','q2_freq']]
test3.rename(columns={'id':'test_id'},inplace=True)
test3=test3[['test_id','q1_freq','q2_freq']]
test3.head()

,test_id,q1_freq,q2_freq
0,0,1,1
1,1,2,2
2,2,1,1
3,3,1,1
4,4,1,1


In [15]:
test_df=pd.merge(test1,test2,how='inner', on='test_id') 
test_df=pd.merge(test_df,test3,how='inner', on='test_id') 
test_df=pd.merge(test_df,test4,how='inner', on='test_id') 
test_df

,test_id,question1,question2,word_match,tfidf_word_match,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio,q1_freq,q2_freq,q1_q2_intersect
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,0.266667,0.274019,4.0,0.333333,2.0,0.142857,1.0,0.076923,1,1,0
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,0.500000,0.480962,5.0,0.555556,1.0,0.083333,0.0,0.000000,2,2,0
2,2,What but is the best way to send money from Ch...,What you send money to China?,0.444444,0.468893,4.0,0.571429,2.0,0.285714,1.0,0.166667,1,1,0
3,3,Which food not emulsifiers?,What foods fibre?,0.000000,0.000000,1.0,0.200000,0.0,0.000000,0.0,0.000000,1,1,0
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,0.800000,1.000000,3.0,0.500000,2.0,0.400000,1.0,0.250000,1,1,0
5,5,How are the two wheeler insurance from Bharti ...,I admire I am considering of buying insurance ...,0.200000,0.222582,1.0,0.111111,0.0,0.000000,0.0,0.000000,1,1,0
6,6,How can I reduce my belly fat through a diet?,How can I reduce my lower belly fat in one month?,0.600000,0.624099,4.0,0.500000,1.0,0.111111,0.0,0.000000,28,30,26
7,7,"By scrapping the 500 and 1000 rupee notes, how...",How will the recent move to declare 500 and 10...,0.363636,0.285686,5.0,0.250000,3.0,0.150000,1.0,0.050000,1,1,0
8,8,What are the how best books of all time?,What are some of the military history books of...,0.571429,0.588009,3.0,0.500000,2.0,0.400000,1.0,0.250000,1,1,0
9,9,After 12th years old boy and I had sex with a ...,Can a 14 old guy date a 12 year old girl?,0.235294,0.220013,4.0,0.235294,1.0,0.050000,0.0,0.000000,1,2,0


In [ ]:
train_df.drop(['qid1','qid2','question1','question2'],axis=1, inplace=True)
test_df.drop(['question1','question2'],axis=1, inplace=True)

In [20]:
#train_df=pd.merge(train1,train2,how='inner', on='id') 
#test_df=pd.merge(test1,test2,how='inner', on='test_id') 

#del train1,train2,test1,test2

print("train_df:",train_df.shape)
print("test_df",test_df.shape)
train_df.head()

train_df: (404290, 13)
test_df (2345796, 12)


,id,is_duplicate,word_match,tfidf_word_match,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio,q1_freq,q2_freq,q1_q2_intersect
0,0,0,0.727273,0.772164,6.0,0.857143,5.0,0.625000,4.0,0.571429,1,2,0
1,1,0,0.307692,0.361758,6.0,0.461538,4.0,0.307692,3.0,0.250000,8,3,0
2,2,0,0.363636,0.355191,3.0,0.300000,0.0,0.000000,0.0,0.000000,2,1,0
3,3,0,0.000000,0.000000,1.0,0.062500,0.0,0.000000,0.0,0.000000,1,1,0
4,4,0,0.000000,0.000000,3.0,0.200000,0.0,0.000000,0.0,0.000000,3,1,0


In [19]:
print("train_df:",train_df.shape)
print("test_df",test_df.shape)
test_df.head()

train_df: (404290, 13)
test_df (2345796, 12)


,test_id,word_match,tfidf_word_match,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio,q1_freq,q2_freq,q1_q2_intersect
0,0,0.266667,0.274019,4.0,0.333333,2.0,0.142857,1.0,0.076923,1,1,0
1,1,0.500000,0.480962,5.0,0.555556,1.0,0.083333,0.0,0.000000,2,2,0
2,2,0.444444,0.468893,4.0,0.571429,2.0,0.285714,1.0,0.166667,1,1,0
3,3,0.000000,0.000000,1.0,0.200000,0.0,0.000000,0.0,0.000000,1,1,0
4,4,0.800000,1.000000,3.0,0.500000,2.0,0.400000,1.0,0.250000,1,1,0


# 训练集多项式组合

In [27]:
from sklearn.preprocessing import MinMaxScaler
train_df.common_unigrams_len = MinMaxScaler().fit_transform(train_df.common_unigrams_len)
train_df.common_bigrams_len = MinMaxScaler().fit_transform(train_df.common_bigrams_len)
train_df.common_trigrams_len = MinMaxScaler().fit_transform(train_df.common_trigrams_len)
train_df.q1_freq = MinMaxScaler().fit_transform(train_df.q1_freq)
train_df.q2_freq = MinMaxScaler().fit_transform(train_df.q2_freq)

E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a s

In [21]:
label=train_df[['id','is_duplicate']]
train_df=train_df.drop(['id','is_duplicate'],axis=1)

In [60]:
train_df.fillna(0,inplace=True)#404290 rows × 11 columns
train_df.head()

,word_match,tfidf_word_match,common_unigrams_len,common_unigrams_ratio,common_bigrams_len,common_bigrams_ratio,common_trigrams_len,common_trigrams_ratio,q1_freq,q2_freq,q1_q2_intersect
0,0.727273,0.772164,0.200000,0.857143,0.135135,0.625000,0.114286,0.571429,0.000000,0.003448,0
1,0.307692,0.361758,0.200000,0.461538,0.108108,0.307692,0.085714,0.250000,0.024138,0.006897,0
2,0.363636,0.355191,0.100000,0.300000,0.000000,0.000000,0.000000,0.000000,0.003448,0.000000,0
3,0.000000,0.000000,0.033333,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,0.000000,0.000000,0.100000,0.200000,0.000000,0.000000,0.000000,0.000000,0.006897,0.000000,0


In [65]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True)
train_df=poly.fit_transform(train_df)
train_df = pd.DataFrame(train_df.reshape(404290,67))
#train_df=pd.concat([label,train_df],axis=1)
train_df
#df.interpolate(axis=1).values.reshape(dataset.shape)

,0,1,2,3,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
0,1.0,0.727273,0.772164,0.200000,0.857143,0.135135,0.625000,0.114286,0.571429,0.000000,...,0.065306,0.000000,0.000394,0.000000,0.000000,0.001970,0.000000,0.000000,0.000000,0.000000
1,1.0,0.307692,0.361758,0.200000,0.461538,0.108108,0.307692,0.085714,0.250000,0.024138,...,0.021429,0.002069,0.000591,0.000000,0.006034,0.001724,0.000000,0.000166,0.000000,0.000000
2,1.0,0.363636,0.355191,0.100000,0.300000,0.000000,0.000000,0.000000,0.000000,0.003448,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1.0,0.000000,0.000000,0.033333,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1.0,0.000000,0.000000,0.100000,0.200000,0.000000,0.000000,0.000000,0.000000,0.006897,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,1.0,0.470588,0.510771,0.166667,0.312500,0.027027,0.050000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,1.0,0.000000,0.000000,0.033333,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,1.0,0.500000,0.645836,0.066667,0.500000,0.027027,0.333333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,1.0,0.500000,0.396755,0.100000,0.375000,0.000000,0.000000,0.000000,0.000000,0.003448,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000024,0.003448,0.006897
9,1.0,0.363636,0.503203,0.133333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [66]:
train_df=pd.concat([label,train_df],axis=1)
train_df

,id,is_duplicate,0,1,2,3,4,5,6,7,...,57,58,59,60,61,62,63,64,65,66
0,0,0,1.0,0.727273,0.772164,0.200000,0.857143,0.135135,0.625000,0.114286,...,0.065306,0.000000,0.000394,0.000000,0.000000,0.001970,0.000000,0.000000,0.000000,0.000000
1,1,0,1.0,0.307692,0.361758,0.200000,0.461538,0.108108,0.307692,0.085714,...,0.021429,0.002069,0.000591,0.000000,0.006034,0.001724,0.000000,0.000166,0.000000,0.000000
2,2,0,1.0,0.363636,0.355191,0.100000,0.300000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,3,0,1.0,0.000000,0.000000,0.033333,0.062500,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,0,1.0,0.000000,0.000000,0.100000,0.200000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,5,1,1.0,0.470588,0.510771,0.166667,0.312500,0.027027,0.050000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,6,0,1.0,0.000000,0.000000,0.033333,0.100000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,7,1,1.0,0.500000,0.645836,0.066667,0.500000,0.027027,0.333333,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,8,0,1.0,0.500000,0.396755,0.100000,0.375000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000024,0.003448,0.006897
9,9,0,1.0,0.363636,0.503203,0.133333,0.333333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# 测试集多项式组合

In [67]:
from sklearn.preprocessing import MinMaxScaler
test_df.common_unigrams_len = MinMaxScaler().fit_transform(test_df.common_unigrams_len)
test_df.common_bigrams_len = MinMaxScaler().fit_transform(test_df.common_bigrams_len)
test_df.common_trigrams_len = MinMaxScaler().fit_transform(test_df.common_trigrams_len)
test_df.q1_freq = MinMaxScaler().fit_transform(test_df.q1_freq)
test_df.q2_freq = MinMaxScaler().fit_transform(test_df.q2_freq)

label=test_df[['test_id']]
test_df=test_df.drop(['test_id'],axis=1)

test_df.fillna(0,inplace=True)#2345796 rows × 11 columns

E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
E:\Anaconda2\envs\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a s

In [69]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(interaction_only=True)
test_df=poly.fit_transform(test_df)
test_df = pd.DataFrame(test_df.reshape(2345796,67))
test_df=pd.concat([label,test_df],axis=1)
test_df

,test_id,0,1,2,3,4,5,6,7,8,...,57,58,59,60,61,62,63,64,65,66
0,0,1.0,0.266667,0.274019,0.114286,0.333333,0.038462,0.142857,0.015625,0.076923,...,0.001202,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000
1,1,1.0,0.500000,0.480962,0.142857,0.555556,0.019231,0.083333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1.387112e-07,0.000000,0.000000
2,2,1.0,0.444444,0.468893,0.114286,0.571429,0.038462,0.285714,0.015625,0.166667,...,0.002604,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000
3,3,1.0,0.000000,0.000000,0.028571,0.200000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000
4,4,1.0,0.800000,1.000000,0.085714,0.500000,0.038462,0.400000,0.015625,0.250000,...,0.003906,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000
5,5,1.0,0.200000,0.222582,0.028571,0.111111,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000
6,6,1.0,0.600000,0.624099,0.114286,0.500000,0.019231,0.111111,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1.086108e-04,0.261453,0.280819
7,7,1.0,0.363636,0.285686,0.142857,0.250000,0.057692,0.150000,0.015625,0.050000,...,0.000781,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000
8,8,1.0,0.571429,0.588009,0.085714,0.500000,0.038462,0.400000,0.015625,0.250000,...,0.003906,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000
9,9,1.0,0.235294,0.220013,0.114286,0.235294,0.019231,0.050000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00,0.000000,0.000000


In [73]:
test_df.drop([0],axis=1,inplace=True)
train_df.drop([0],axis=1,inplace=True)

# *Rebalancing the Data*

In [74]:
y_train = train_df['is_duplicate'].values
pos_train = train_df[y_train == 1]
neg_train = train_df[y_train == 0]


# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

x_train = pd.concat([pos_train, neg_train])
y_train = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()
del pos_train, neg_train

print("train_df:",train_df.shape)
print("test_df",test_df.shape)

print("x_train:",x_train.shape)
print("y_train",len(y_train))

0.19124366100096607
train_df: (404290, 68)
test_df (2345796, 67)
x_train: (780486, 68)
y_train 780486


In [75]:
x_train.drop(['id'],axis=1, inplace=True)
sub = pd.DataFrame()
sub['test_id'] = test_df['test_id']
x_test=test_df.drop(['test_id'],axis=1)

#sub['is_duplicate'] = p_test

# XGBoost

In [77]:
# Finally, we split some of the data off for validation
from sklearn.cross_validation import train_test_split
#x_train.drop(['id','is_duplicate'],axis=1, inplace=True)

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=4242)
print("x_train:",x_train.shape)
print("y_train",len(y_train))
print("x_valid:",x_valid.shape)
print("y_valid",len(y_valid))

print("x_test:",x_test.shape)

x_train: (624388, 67)
y_train 624388
x_valid: (156098, 67)
y_valid 156098
x_test: (2345796, 66)


In [78]:
x_train.drop(['is_duplicate'],axis=1, inplace=True)
x_valid.drop(['is_duplicate'],axis=1, inplace=True)

E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
E:\Anaconda2\envs\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [80]:
import xgboost as xgb

# Set our parameters for xgboost
#params = {}
#params['objective'] = 'binary:logistic'
#params['eval_metric'] = 'logloss'
#params['eta'] = 0.02
#params['max_depth'] = 6

#XGBoost自带接口
params={
    'eta': 0.1,
    'max_depth':6,   
    'min_child_weight':1,
    'gamma':0.3, 
    'subsample':0.8,
    'colsample_bytree':0.8,
    'booster':'gbtree',
    'objective': 'binary:logistic',
    'nthread':12,
    'scale_pos_weight': 1,
    'lambda':1,  
    'seed':27,
    'silent':0 ,
    'eval_metric': 'logloss'
}

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(x_test)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

bst = xgb.train(params, d_train, 3000, watchlist, early_stopping_rounds=500, verbose_eval=10)

[0]	train-logloss:0.624325	valid-logloss:0.624601
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 500 rounds.
[10]	train-logloss:0.324125	valid-logloss:0.326072
[20]	train-logloss:0.247895	valid-logloss:0.250744
[30]	train-logloss:0.223877	valid-logloss:0.227349
[40]	train-logloss:0.215273	valid-logloss:0.219204
[50]	train-logloss:0.211494	valid-logloss:0.215765
[60]	train-logloss:0.20928	valid-logloss:0.213839
[70]	train-logloss:0.208074	valid-logloss:0.212815
[80]	train-logloss:0.206783	valid-logloss:0.211766
[90]	train-logloss:0.205723	valid-logloss:0.211022
[100]	train-logloss:0.204791	valid-logloss:0.210322
[110]	train-logloss:0.203944	valid-logloss:0.209766
[120]	train-logloss:0.203267	valid-logloss:0.209366
[130]	train-logloss:0.202651	valid-logloss:0.209019
[140]	train-logloss:0.202075	valid-logloss:0.208701
[150]	train-logloss:0.201416	valid-logloss:0.208311
[160]	train-logloss:0.200824

In [85]:
bst.best_iteration

2997

In [81]:
y_bst= bst.predict(d_test)
sub['is_duplicate'] = y_bst
sub

,test_id,is_duplicate
0,0,0.017364
1,1,0.180221
2,2,0.298109
3,3,0.001773
4,4,0.065461
5,5,0.000172
6,6,0.999996
7,7,0.106918
8,8,0.350059
9,9,0.001497


In [82]:
sub.to_csv('../output/xgb_20170521_A.csv', index=False)

# Ensemble

In [ ]:
output1=pd.read_csv('../output/0.2613_lstm_215_108_0.19_0.20.csv')
output2=pd.read_csv('../output/0.2662_lstm_232_118_0.39_0.15.csv')
output3=pd.read_csv('../output/0.2713_lstm_208_112_0.39_0.27.csv')
output4=pd.read_csv('../output/0.2723_lstm_190_136_0.29_0.27.csv')

output=output1[['test_id']]

x1=output1.is_duplicate
x2=output2.is_duplicate
x3=output3.is_duplicate
x4=output4.is_duplicate

output['is_duplicate']=0.25*x1+0.25*x2+0.25*x3+0.25*x4
output.to_csv('../output/Avg_20170521_A.csv', index=False)#0.28687

# Explore

In [49]:
def word1(que):
    return [word for word in word_tokenize(que.lower()) if word not in eng_stopwords]#word_tokenize:对句子进行分词

def word2(que):
    return [word for word in que.lower().split() if word not in eng_stopwords]#:对句子进行分词

def get_bigrams(que):
    return [i for i in ngrams(word_tokenize(que.lower()), 2)]#组合分词

In [50]:
sentence1 = """At eight o'clock on Thursday morning Arthur didn't feel very good?"""
sentence2 = """At eight o'clock on Thursday morning Arthur feel very good!"""
x1= word1(sentence1)
x2= word1(sentence2)

y1= word2(sentence1)
y2= word2(sentence2)

z1= get_bigrams(sentence1)
z2= get_bigrams(sentence2)

print(x1)
print(x2)
print("  ")
print(y1)
print(y2)
print("  ")
print(z1)
print(z2)


['eight', "o'clock", 'thursday', 'morning', 'arthur', "n't", 'feel', 'good', '?']
['eight', "o'clock", 'thursday', 'morning', 'arthur', 'feel', 'good', '!']
  
['eight', "o'clock", 'thursday', 'morning', 'arthur', "didn't", 'feel', 'good?']
['eight', "o'clock", 'thursday', 'morning', 'arthur', 'feel', 'good!']
  
[('at', 'eight'), ('eight', "o'clock"), ("o'clock", 'on'), ('on', 'thursday'), ('thursday', 'morning'), ('morning', 'arthur'), ('arthur', 'did'), ('did', "n't"), ("n't", 'feel'), ('feel', 'very'), ('very', 'good'), ('good', '?')]
[('at', 'eight'), ('eight', "o'clock"), ("o'clock", 'on'), ('on', 'thursday'), ('thursday', 'morning'), ('morning', 'arthur'), ('arthur', 'feel'), ('feel', 'very'), ('very', 'good'), ('good', '!')]


In [51]:
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

porter = PorterStemmer()
snowball = SnowballStemmer('english')

def stem_str(x,stemmer=SnowballStemmer('english')):
    x = text.re.sub("[^a-zA-Z0-9]"," ", x)
    x = (" ").join([stemmer.stem(z) for z in x.split(" ")])
    x = " ".join(x.split())
    return x

In [55]:
print(sentence1)
print(stem_str(sentence1,porter))
print(sentence2)
print(stem_str(sentence2,porter))


At eight o'clock on Thursday morning Arthur didn't feel very good?
At eight o clock on Thursday morn Arthur didn t feel veri good
At eight o'clock on Thursday morning Arthur feel very good!
At eight o clock on Thursday morn Arthur feel veri good
